In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import yaml
import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
import tensorflow as tf
from pathlib import Path
from time import strftime
from shutil import rmtree
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib
#matplotlib.use("TKAgg", force=True)
%matplotlib inline
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from rtapipe.lib.utils.misc import dotdict
from rtapipe.lib.dataset.data_manager import DataManager
from rtapipe.lib.datasource.Photometry3 import OnlinePhotometry, SimulationParams
from rtapipe.lib.plotting.plotting import plot_sequences
from rtapipe.lib.models.anomaly_detector_builder import AnomalyDetectorBuilder
from rtapipe.lib.evaluation.pval import get_pval_table, get_threshold_for_sigma
from rtapipe.lib.standardanalysis.li_ma import LiMa
from rtapipe.lib.evaluation.pval import get_pval_table, get_threshold_for_sigma, get_sigma_from_pvalue, get_sigma_for_ts_array, get_sigma_from_ts

# Loading the models

In [3]:
def load_model(model_id):
    with open("./trained_models.yaml", "r") as f:
        try:
            configs = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
    model_config = [c for c in configs["models"] if c["id"] == model_id]
    model_config = dotdict(model_config.pop())
    model_config.ad = AnomalyDetectorBuilder.getAnomalyDetector(name=model_config.name, timesteps=model_config.timesteps, nfeatures=model_config.nfeatures, load_model="True", training_epoch_dir=model_config.path, training=False)
    model_config.pvalue_table = get_pval_table(model_config.pval_path) 
    return model_config

In [28]:
model_config_cnn = load_model(6)
model_config_cnn

AnomalyDetector_cnn_l2_u32 - input shape: (5,3)


{'id': 6,
 'patience': 5,
 'integrationtime': 1,
 'timesteps': 5,
 'nfeatures': 3,
 'scaler': 'minmax',
 'name': 'AnomalyDetector_cnn_l2_u32',
 'path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221220-224157_it_1/model_AnomalyDetector_cnn_l2_u32_dataset_train_itime_1_a_tsl_5_nbins_3_tsl_500/epochs/epoch_5',
 'epoch': 5,
 'pval_path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221220-224157_it_1/model_AnomalyDetector_cnn_l2_u32_dataset_train_itime_1_a_tsl_5_nbins_3_tsl_500/epochs/epoch_5/pvalues/pval_20221226-110743/pvalue_bins_100_0.numpy.txt',
 'scaler_path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221220-224157_it_1/model_AnomalyDetector_cnn_l2_u32_dataset_train_itime_1_a_tsl_5_nbins_3_tsl_500/fitted_scaler_minmax_itime_1_tsl_500.pickle',
 'ad': <rtapipe.lib.models.anomaly_detector_cnn.AnomalyDetector_cnn_l2_u32 at 0x2b4aa96280d0>,
 'pvalue_table':     threshold  threshold_err        pvalue    pvalue_err  sigma
 0    0.002418       0.000158  1.269

In [5]:
model_config_rnn = load_model(7)
model_config_rnn

AnomalyDetector_rnn_l2_u32 - input shape: (5,3)


{'id': 7,
 'patience': 5,
 'integrationtime': 1,
 'timesteps': 5,
 'nfeatures': 3,
 'scaler': 'minmax',
 'name': 'AnomalyDetector_rnn_l2_u32',
 'path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221220-224157_it_1/model_AnomalyDetector_rnn_l2_u32_dataset_train_itime_1_a_tsl_5_nbins_3_tsl_500/epochs/epoch_5',
 'epoch': 5,
 'pval_path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221220-224157_it_1/model_AnomalyDetector_rnn_l2_u32_dataset_train_itime_1_a_tsl_5_nbins_3_tsl_500/epochs/epoch_5/pvalues/pval_20221222-114342/pvalue_bins_100_0.numpy.txt',
 'scaler_path': '/data01/homes/baroncelli/phd/rtapipe/notebooks/run_20221220-224157_it_1/model_AnomalyDetector_rnn_l2_u32_dataset_train_itime_1_a_tsl_5_nbins_3_tsl_500/fitted_scaler_minmax_itime_1_tsl_500.pickle',
 'ad': <rtapipe.lib.models.anomaly_detector_rnn.AnomalyDetector_rnn_l2_u32 at 0x2b483445b0a0>,
 'pvalue_table':     threshold  threshold_err        pvalue    pvalue_err  sigma
 0    0.000201       0.000015  1.339

# Loading the test dataset

In [6]:
output_dir = "./logs/test_models_dataset_itime_1_e_out"

In [7]:
features_names = ["EB_0.04-0.117","EB_2-0.117-0.342","EB_0.342-1"]

In [8]:
data_manager = DataManager(output_dir)

In [9]:
dataset_id="test_itime_1_c_tsl_5_nbins_3"

In [10]:
dataset_folder = "/data01/homes/baroncelli/phd/rtapipe/scripts/ml/dataset_generation/test/itime_5_e/fits_data"
fits_files = DataManager.load_fits_data(dataset_folder, limit=500)
test_set_size = len(fits_files)

Loaded 188 files


In [11]:
sim_params = SimulationParams(runid=None, onset=250, emin=0.04, emax=1, tmin=0, tobs=500, offset=0.5, irf="North_z40_5h_LST", roi=2.5, caldb="prod5-v0.1", simtype="grb")
multiple_templates = True
add_target_region = True
integration_time = 1
number_of_energy_bins = 3
tsl = 500
threads = 30
normalize = True
#data_manager.transform_to_timeseries(fits_files, sim_params, add_target_region, integration_time=integration_time, number_of_energy_bins=number_of_energy_bins, tsl=tsl, normalize=normalize, threads=threads, multiple_templates=multiple_templates)
data_manager.load_saved_data(1, 5)

[2022-12-26 11:11:32.968121] Loaded data from logs/test_models_dataset_itime_1_e_out/data_cache. Loaded 188 templates.


In [12]:
assert len(data_manager.data.keys()) == test_set_size

## Plot some samples

In [13]:
for template in list(data_manager.data.keys())[0:5]:
    data_manager.plot_timeseries(template, data_manager.data[template], 1, sim_params, output_dir, labels=features_names)

Produced: logs/test_models_dataset_itime_1_e_out/template_run0002_ID000044_trial_0_2022-12-26 11:11:33.024770.png
Saved: ./logs/test_models_dataset_itime_1_e_out/template_run0002_ID000044_trial_0_2022-12-26 11:11:33.024770.png
Produced: logs/test_models_dataset_itime_1_e_out/template_run0011_ID000139_trial_0_2022-12-26 11:11:35.944904.png
Saved: ./logs/test_models_dataset_itime_1_e_out/template_run0011_ID000139_trial_0_2022-12-26 11:11:35.944904.png
Produced: logs/test_models_dataset_itime_1_e_out/template_run0016_ID000340_trial_0_2022-12-26 11:11:38.430030.png
Saved: ./logs/test_models_dataset_itime_1_e_out/template_run0016_ID000340_trial_0_2022-12-26 11:11:38.430030.png
Produced: logs/test_models_dataset_itime_1_e_out/template_run0017_ID000132_trial_0_2022-12-26 11:11:40.831226.png
Saved: ./logs/test_models_dataset_itime_1_e_out/template_run0017_ID000132_trial_0_2022-12-26 11:11:40.831226.png
Produced: logs/test_models_dataset_itime_1_e_out/template_run0017_ID000261_trial_0_2022-12-2

# TESTING

In [14]:
data_manager.load_scaler(model_config_rnn.scaler_path) # model_config_rnn.scaler_path or model_config_cnn.scaler_path

In [15]:
test_all_x, test_all_y = data_manager.get_test_set_all_templates(verbose=True, onset=250, integration_time=integration_time, sub_window_size=5, stride=1)

Pivot index:  251
windows_before_pivot:  (247, 5, 3)
windows_after_pivot:  (249, 5, 3)
[2022-12-26 11:11:45.719425] Loaded 1 timeseries from template run0002_ID000044.
Single file shape before sub-windowing: (500, 3). Single file shape after sub-windowing: (496, 5, 3)
[2022-12-26 11:11:45.719442] test_x shape: (496, 5, 3)
[2022-12-26 11:11:45.719449] test_y shape: (496,)
Pivot index:  251
windows_before_pivot:  (247, 5, 3)
windows_after_pivot:  (249, 5, 3)
[2022-12-26 11:11:45.720024] Loaded 1 timeseries from template run0011_ID000139.
Single file shape before sub-windowing: (500, 3). Single file shape after sub-windowing: (496, 5, 3)
[2022-12-26 11:11:45.720037] test_x shape: (496, 5, 3)
[2022-12-26 11:11:45.720043] test_y shape: (496,)
Pivot index:  251
windows_before_pivot:  (247, 5, 3)
windows_after_pivot:  (249, 5, 3)
[2022-12-26 11:11:45.720621] Loaded 1 timeseries from template run0016_ID000340.
Single file shape before sub-windowing: (500, 3). Single file shape after sub-window

windows_before_pivot:  (247, 5, 3)
windows_after_pivot:  (249, 5, 3)
[2022-12-26 11:11:45.897217] Loaded 1 timeseries from template run0607_ID000306.
Single file shape before sub-windowing: (500, 3). Single file shape after sub-windowing: (496, 5, 3)
[2022-12-26 11:11:45.897232] test_x shape: (496, 5, 3)
[2022-12-26 11:11:45.897239] test_y shape: (496,)
Pivot index:  251
windows_before_pivot:  (247, 5, 3)
windows_after_pivot:  (249, 5, 3)
[2022-12-26 11:11:45.899216] Loaded 1 timeseries from template run0610_ID000260.
Single file shape before sub-windowing: (500, 3). Single file shape after sub-windowing: (496, 5, 3)
[2022-12-26 11:11:45.899230] test_x shape: (496, 5, 3)
[2022-12-26 11:11:45.899237] test_y shape: (496,)
Pivot index:  251
windows_before_pivot:  (247, 5, 3)
windows_after_pivot:  (249, 5, 3)
[2022-12-26 11:11:45.901309] Loaded 1 timeseries from template run0615_ID000207.
Single file shape before sub-windowing: (500, 3). Single file shape after sub-windowing: (496, 5, 3)
[

# RNN

## Standard metrics - 3 sigma threshold

In [16]:
SIGMA_THRESHOLD = 3

In [17]:
sigma_threshold = get_threshold_for_sigma(model_config_rnn.pvalue_table, SIGMA_THRESHOLD)
model_config_rnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_rnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_rnn.pvalue_table, model_config_rnn.ad.threshold)} sigma")

Threshold: 0.000806065107 corresponding to 3.0062598241 sigma


In [18]:
print(f"************** Evaluating {model_config_rnn.name} patience={model_config_rnn.patience} **************")
metrics = model_config_rnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_rnn.ad.detection_delay(test_all_y, model_config_rnn.ad.predict(test_all_x), test_set_size, model_config_rnn.timesteps))

************** Evaluating AnomalyDetector_rnn_l2_u32 patience=5 **************
Using threshold 0.000806065107
accuracy 0.7425896533973919
precision 0.9992995052756009
recall 0.4875886524822695
f1 0.6553917275637805
roc_auc 0.7436220461136475
confusion_matrix [[46420, 16], [23987, 22825]]
false_positive_rate 0.0007004947243991069
detection_delay: 34.819148936170215


## Standard metrics - 5 sigma threshold

In [19]:
SIGMA_THRESHOLD = 5

In [20]:
sigma_threshold = get_threshold_for_sigma(model_config_rnn.pvalue_table, SIGMA_THRESHOLD)
model_config_rnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_rnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_rnn.pvalue_table, model_config_rnn.ad.threshold)} sigma")

Threshold: 0.002622639387 corresponding to 5.0410478147 sigma


In [21]:
print(f"************** Evaluating {model_config_rnn.name} patience={model_config_rnn.patience} **************")
metrics = model_config_rnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_rnn.ad.detection_delay(test_all_y, model_config_rnn.ad.predict(test_all_x), test_set_size, model_config_rnn.timesteps))

************** Evaluating AnomalyDetector_rnn_l2_u32 patience=5 **************
Using threshold 0.002622639387
accuracy 0.6612474262182567
precision 0.9998030203545634
recall 0.32527984277535676
f1 0.49086102962509276
roc_auc 0.6626076188637745
confusion_matrix [[46433, 3], [31585, 15227]]
false_positive_rate 0.0001969796454366382
detection_delay: 33.005319148936174


## Sigma table

In [22]:
def crop_to_5(s):
    if s > 5: 
        s = 5.00001
    return s

In [23]:
data = {}
for template in tqdm(list(data_manager.data.keys())):
    test_x, test_y = data_manager.get_test_set(template=template, onset=250, integration_time=1, sub_window_size=5, stride=1)
    model_config_rnn.ad.predict(test_x)
    data[template] = get_sigma_for_ts_array(model_config_rnn.pvalue_table, model_config_rnn.ad.get_reconstruction_errors(), verbose=False)

sigma_table_rnn = pd.DataFrame(data, index=DataManager.get_tbin_of_sequences(496, 1, 5, 1))    
sigma_table_rnn = sigma_table_rnn.applymap(crop_to_5)

100%|█████████████████████████████████████████| 188/188 [00:48<00:00,  3.91it/s]


AttributeError: 'str' object has no attribute 'joinpath'

In [24]:
sigma_table_rnn.to_csv(Path(output_dir).joinpath("sigma_table_rnn_itime_1_dataset_e.csv"))

In [25]:
sigma_table_rnn

,run0002_ID000044,run0011_ID000139,run0016_ID000340,run0017_ID000132,run0017_ID000261,run0019_ID000070,run0019_ID000268,run0026_ID000127,run0029_ID000397,run0029_ID000446,run0032_ID000190,run0038_ID000228,run0043_ID000231,run0043_ID000324,run0043_ID000460,run0046_ID000029,run0046_ID000300,run0048_ID000465,run0051_ID000166,run0051_ID000434,run0055_ID000128,run0066_ID000041,run0067_ID000395,run0076_ID000465,run0079_ID000383,run0096_ID000038,run0099_ID000496,run0101_ID000172,run0109_ID000136,run0111_ID000027,run0113_ID000112,run0114_ID000299,run0115_ID000189,run0118_ID000382,run0118_ID000393,run0121_ID000156,run0130_ID000461,run0138_ID000218,run0142_ID000123,run0146_ID000301,run0147_ID000002,run0154_ID000068,run0155_ID000328,run0157_ID000407,run0158_ID000220,run0158_ID000313,run0163_ID000414,run0173_ID000131,run0176_ID000473,run0180_ID000255,run0181_ID000296,run0182_ID000054,run0191_ID000005,run0192_ID000017,run0192_ID000068,run0193_ID000134,run0197_ID000145,run0199_ID000215,run0199_ID000278,run0210_ID000082,run0214_ID000101,run0218_ID000245,run0221_ID000122,run0222_ID000318,run0226_ID000257,run0231_ID000152,run0231_ID000320,run0233_ID000376,run0240_ID000371,run0242_ID000066,run0252_ID000066,run0254_ID000456,run0258_ID000139,run0258_ID000165,run0278_ID000152,run0283_ID000065,run0284_ID000362,run0286_ID000021,run0286_ID000099,run0291_ID000001,run0294_ID000203,run0301_ID000376,run0309_ID000061,run0315_ID000129,run0317_ID000219,run0318_ID000143,run0325_ID000165,run0339_ID000241,run0340_ID000060,run0341_ID000126,run0343_ID000134,run0344_ID000129,run0349_ID000154,run0350_ID000189,run0352_ID000076,run0354_ID000089,run0358_ID000063,run0363_ID000327,run0369_ID000392,run0370_ID000146,run0382_ID000397,run0383_ID000191,run0385_ID000009,run0395_ID000138,run0398_ID000008,run0403_ID000285,run0406_ID000126,run0407_ID000101,run0410_ID000425,run0421_ID000212,run0423_ID000024,run0428_ID000157,run0429_ID000078,run0431_ID000202,run0436_ID000045,run0444_ID000110,run0452_ID000030,run0453_ID000359,run0457_ID000240,run0459_ID000351,run0460_ID000344,run0461_ID000109,run0467_ID000192,run0478_ID000180,run0478_ID000222,run0478_ID000299,run0481_ID000384,run0493_ID000305,run0494_ID000343,run0496_ID000386,run0498_ID000059,run0498_ID000071,run0498_ID000373,run0501_ID000313,run0501_ID000398,run0508_ID000316,run0524_ID000291,run0533_ID000171,run0535_ID000181,run0556_ID000312,run0559_ID000332,run0562_ID000069,run0563_ID000044,run0569_ID000136,run0576_ID000397,run0583_ID000329,run0584_ID000213,run0586_ID000161,run0592_ID000035,run0607_ID000306,run0610_ID000260,run0615_ID000207,run0620_ID000109,run0629_ID000224,run0637_ID000089,run0641_ID000127,run0641_ID000226,run0642_ID000033,run0652_ID000307,run0655_ID000259,run0658_ID000260,run0667_ID000126,run0688_ID000044,run0697_ID000167,run0709_ID000364,run0711_ID000203,run0712_ID000163,run0722_ID000130,run0722_ID000187,run0762_ID000202,run0763_ID000224,run0765_ID000033,run0767_ID000231,run0780_ID000007,run0788_ID000236,run0792_ID000290,run0793_ID000465,run0798_ID000122,run0798_ID000403,run0799_ID000140,run0804_ID000031,run0811_ID000253,run0812_ID000337,run0839_ID000040,run0853_ID000363,run0879_ID000213,run0880_ID000380,run0881_ID000086
0-5,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.684152,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,1.416807,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,1.343082,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,1.587517,1.370978,0.000000,0.000000,0.00000

# CNN

## Standard metrics - 3 sigma

In [29]:
SIGMA_THRESHOLD = 3

In [30]:
sigma_threshold = get_threshold_for_sigma(model_config_cnn.pvalue_table, SIGMA_THRESHOLD)
model_config_cnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_cnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_cnn.pvalue_table, model_config_cnn.ad.threshold)} sigma")

Threshold: 0.0090684931345 corresponding to 3.0057392313 sigma


In [31]:
print(f"************** Evaluating {model_config_cnn.name} patience={model_config_cnn.patience} **************")
metrics = model_config_cnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_cnn.ad.detection_delay(test_all_y, model_config_cnn.ad.predict(test_all_x), test_set_size, model_config_cnn.timesteps))

************** Evaluating AnomalyDetector_cnn_l2_u32 patience=5 **************
Using threshold 0.0090684931345
accuracy 0.7069856726149623
precision 0.9996923234705912
recall 0.41645304622746304
f1 0.587969176481233
roc_auc 0.7081619180659237
confusion_matrix [[46430, 6], [27317, 19495]]
false_positive_rate 0.0003076765294087483
detection_delay: 38.13297872340426


## Standard metrics - 5 sigma

In [32]:
SIGMA_THRESHOLD = 5

In [33]:
sigma_threshold = get_threshold_for_sigma(model_config_cnn.pvalue_table, SIGMA_THRESHOLD)
model_config_cnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_cnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_cnn.pvalue_table, model_config_cnn.ad.threshold)} sigma")

Threshold: 0.0309210772074999 corresponding to 5.0266484899 sigma


In [34]:
print(f"************** Evaluating {model_config_cnn.name} patience={model_config_cnn.patience} **************")
metrics = model_config_cnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_cnn.ad.detection_delay(test_all_y, model_config_cnn.ad.predict(test_all_x), test_set_size, model_config_cnn.timesteps))

************** Evaluating AnomalyDetector_cnn_l2_u32 patience=5 **************
Using threshold 0.0309210772074999
accuracy 0.6297614962251201
precision 0.9999186330349878
recall 0.2625181577373323
f1 0.4158573313931846
roc_auc 0.6312483113606983
confusion_matrix [[46435, 1], [34523, 12289]]
false_positive_rate 8.136696501220504e-05
detection_delay: 18.04787234042553


## Sigma table

In [37]:
data = {}
for template in tqdm(list(data_manager.data.keys())):
    test_x, test_y = data_manager.get_test_set(template=template, onset=250, integration_time=1, sub_window_size=5, stride=1)
    model_config_cnn.ad.predict(test_x)
    data[template] = get_sigma_for_ts_array(model_config_cnn.pvalue_table, model_config_cnn.ad.get_reconstruction_errors(), verbose=False)

sigma_table_cnn = pd.DataFrame(data, index=DataManager.get_tbin_of_sequences(496, 1, 5, 1))    
sigma_table_cnn = sigma_table_cnn.applymap(crop_to_5)

100%|█████████████████████████████████████████| 188/188 [00:45<00:00,  4.12it/s]


In [38]:
sigma_table_cnn.to_csv(Path(output_dir).joinpath("sigma_table_cnn_itime_1_dataset_e.csv"))

In [39]:
sigma_table_cnn

,run0002_ID000044,run0011_ID000139,run0016_ID000340,run0017_ID000132,run0017_ID000261,run0019_ID000070,run0019_ID000268,run0026_ID000127,run0029_ID000397,run0029_ID000446,run0032_ID000190,run0038_ID000228,run0043_ID000231,run0043_ID000324,run0043_ID000460,run0046_ID000029,run0046_ID000300,run0048_ID000465,run0051_ID000166,run0051_ID000434,run0055_ID000128,run0066_ID000041,run0067_ID000395,run0076_ID000465,run0079_ID000383,run0096_ID000038,run0099_ID000496,run0101_ID000172,run0109_ID000136,run0111_ID000027,run0113_ID000112,run0114_ID000299,run0115_ID000189,run0118_ID000382,run0118_ID000393,run0121_ID000156,run0130_ID000461,run0138_ID000218,run0142_ID000123,run0146_ID000301,run0147_ID000002,run0154_ID000068,run0155_ID000328,run0157_ID000407,run0158_ID000220,run0158_ID000313,run0163_ID000414,run0173_ID000131,run0176_ID000473,run0180_ID000255,run0181_ID000296,run0182_ID000054,run0191_ID000005,run0192_ID000017,run0192_ID000068,run0193_ID000134,run0197_ID000145,run0199_ID000215,run0199_ID000278,run0210_ID000082,run0214_ID000101,run0218_ID000245,run0221_ID000122,run0222_ID000318,run0226_ID000257,run0231_ID000152,run0231_ID000320,run0233_ID000376,run0240_ID000371,run0242_ID000066,run0252_ID000066,run0254_ID000456,run0258_ID000139,run0258_ID000165,run0278_ID000152,run0283_ID000065,run0284_ID000362,run0286_ID000021,run0286_ID000099,run0291_ID000001,run0294_ID000203,run0301_ID000376,run0309_ID000061,run0315_ID000129,run0317_ID000219,run0318_ID000143,run0325_ID000165,run0339_ID000241,run0340_ID000060,run0341_ID000126,run0343_ID000134,run0344_ID000129,run0349_ID000154,run0350_ID000189,run0352_ID000076,run0354_ID000089,run0358_ID000063,run0363_ID000327,run0369_ID000392,run0370_ID000146,run0382_ID000397,run0383_ID000191,run0385_ID000009,run0395_ID000138,run0398_ID000008,run0403_ID000285,run0406_ID000126,run0407_ID000101,run0410_ID000425,run0421_ID000212,run0423_ID000024,run0428_ID000157,run0429_ID000078,run0431_ID000202,run0436_ID000045,run0444_ID000110,run0452_ID000030,run0453_ID000359,run0457_ID000240,run0459_ID000351,run0460_ID000344,run0461_ID000109,run0467_ID000192,run0478_ID000180,run0478_ID000222,run0478_ID000299,run0481_ID000384,run0493_ID000305,run0494_ID000343,run0496_ID000386,run0498_ID000059,run0498_ID000071,run0498_ID000373,run0501_ID000313,run0501_ID000398,run0508_ID000316,run0524_ID000291,run0533_ID000171,run0535_ID000181,run0556_ID000312,run0559_ID000332,run0562_ID000069,run0563_ID000044,run0569_ID000136,run0576_ID000397,run0583_ID000329,run0584_ID000213,run0586_ID000161,run0592_ID000035,run0607_ID000306,run0610_ID000260,run0615_ID000207,run0620_ID000109,run0629_ID000224,run0637_ID000089,run0641_ID000127,run0641_ID000226,run0642_ID000033,run0652_ID000307,run0655_ID000259,run0658_ID000260,run0667_ID000126,run0688_ID000044,run0697_ID000167,run0709_ID000364,run0711_ID000203,run0712_ID000163,run0722_ID000130,run0722_ID000187,run0762_ID000202,run0763_ID000224,run0765_ID000033,run0767_ID000231,run0780_ID000007,run0788_ID000236,run0792_ID000290,run0793_ID000465,run0798_ID000122,run0798_ID000403,run0799_ID000140,run0804_ID000031,run0811_ID000253,run0812_ID000337,run0839_ID000040,run0853_ID000363,run0879_ID000213,run0880_ID000380,run0881_ID000086
0-5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,1.522248,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.00000,0.0,0.000000,0.00000,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,

# Li&Ma

In [ ]:
def detect(strategy, integration_time, stride, temporal_bins, tobs, sigma_gt, get_first, region_radius, erange, pht_list=None):

    if pht_list is None:
        raise ValueError("No pht_list provided")

    if strategy not in ["binned", "cumulative"]:
        raise ValueError(f"Unknown Li&Ma strategy: {strategy}")

    pointing = get_obs_pointing(pht_list)
    template = DataManager.extract_runid_from_name(pht_list)
    target = OnlinePhotometry.get_target(Path(os.environ["DATA"]).joinpath("templates", "grb_afterglow", "GammaCatalogV1.0", f"{template}.fits"))
    target['rad'] = region_radius

    phm = Photometrics({'events_filename': pht_list})
    off_regions = Photometrics.find_off_regions('cross', target, pointing, target['rad'])

    data = []
    if strategy == "binned":
        if len(temporal_bins) > 0:
            _temporal_bins = [(float(t.split("-")[0]), float(t.split("-")[1])) for t in temporal_bins]
            for tb in _temporal_bins:
                on, off, alpha, excess, sigma, err_note = phm.counting(src=target, rad=target['rad'], off_regions=off_regions, e_min=erange[0], e_max=erange[1], t_min=tb[0], t_max=tb[1], draconian=False)
                data.append([tb[0], tb[1], on, off, alpha, excess, sigma, err_note])
        else:
            t_min = 0
            while t_min + stride <= tobs:
                on, off, alpha, excess, sigma, err_note = phm.counting(src=target, rad=target['rad'], off_regions=off_regions, e_min=erange[0], e_max=erange[1], t_min=t_min, t_max=t_min + integration_time, draconian=False)
                data.append([t_min, t_min + integration_time, on, off, alpha, excess, sigma, err_note])
                t_min += stride

    elif strategy == "cumulative":
        for tmax in range(0, tobs, 5):
            on, off, alpha, excess, sigma, err_note = phm.counting(src=target, rad=target['rad'], off_regions=off_regions, e_min=erange[0], e_max=erange[1], t_min=0, t_max=tmax, draconian=False)
            data.append([0, tmax, on, off, alpha, excess, sigma, err_note])

    df = pd.DataFrame(columns=["tmin", "tmax", "on", "off", "alpha", "excess", "sigma", "err_note"], data=data)
    if sigma_gt > 0:
        df = df[df["sigma"] > sigma_gt]

    if df.shape[0]>0 and get_first:
        return df.iloc[0]

    return df


In [ ]:
from time import time
import multiprocessing
from functools import partial

from sagsci.tools.utils import get_obs_pointing
from sagsci.wrappers.rtaph.photometry import Photometrics

In [ ]:
sigma_table_rnn.index.values.shape

In [ ]:
li_ma_data = {}
#templates = list(data_manager.data.keys())
#templates = templates[0:10]
templates = []

LiMa_detect_p = partial(detect, "binned", integration_time, 5, sigma_table_rnn.index.values, 500, 0, False, 0.2, [0.03, 1])
start = time()
with multiprocessing.Pool() as p:
    values = p.map(LiMa_detect_p, [DataManager.get_fits_from_template(fits_files, template) for template in templates])

for i, template in enumerate(templates):
     li_ma_data[template] = values[i]["sigma"].values   

print(f"Took {time()-start} s")
    
#
lima_sigma_table = pd.DataFrame(li_ma_data, index=sigma_table_rnn.index.values)
if len(templates) > 0:
    lima_sigma_table.to_csv(Path(output_dir).joinpath("li_ma_detection_table_itime_1_dataset_e.csv"))    

In [ ]:
lima_sigma_table = pd.read_csv(Path(output_dir).joinpath("li_ma_detection_table_itime_1_dataset_e.csv"), index_col=0)

In [ ]:
### FIX INDEPENDENCE
for time_bin in lima_sigma_table.index:
    if int(time_bin.split("-")[1])%5 != 0:
        lima_sigma_table.loc[time_bin] = 0
lima_sigma_table

In [ ]:
lima_sigma_table = lima_sigma_table.applymap(crop_to_5)
print(lima_sigma_table.shape)
print(sigma_table_rnn.shape)
lima_sigma_table.iloc[250:260]

## Results

In [ ]:
templates_detections = dotdict({})
for templ_det in lima_sigma_table:
    templates_detections[templ_det] = dotdict({}) 
    templates_detections[templ_det]["li_ma"] = dotdict({})
    templates_detections[templ_det]["cnn"] = dotdict({})
    templates_detections[templ_det]["rnn"] = dotdict({})

In [ ]:
for templ_det in lima_sigma_table:
    detections_3s = lima_sigma_table[templ_det] >= 3
    detections_5s = lima_sigma_table[templ_det] >= 5
    templates_detections[templ_det]["li_ma"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["li_ma"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["li_ma"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["li_ma"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]

for templ_det in sigma_table_rnn:
    detections_3s = sigma_table_rnn[templ_det] >= 3
    detections_5s = sigma_table_rnn[templ_det] >= 5
    templates_detections[templ_det]["rnn"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["rnn"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["rnn"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["rnn"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]
    
for templ_det in sigma_table_cnn:
    detections_3s = sigma_table_cnn[templ_det] >= 3
    detections_5s = sigma_table_cnn[templ_det] >= 5
    templates_detections[templ_det]["cnn"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["cnn"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["cnn"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["cnn"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]    

In [ ]:
def get_dd(model, templates_detections, sigma, onset_index=45):
    first_detection_indexes = []
    for template_result in templates_detections.values():
        if len(template_result[model][f"{sigma}s_detections_indexes"]) > 0:
            #print(onset_index)
            #print(template_result[model][f"{sigma}s_detections_indexes"])
            first_detection_indexes.append(template_result[model][f"{sigma}s_detections_indexes"][0])
    return round((np.array(first_detection_indexes).mean()-onset_index)*integration_time,2)

In [ ]:
def get_first_detections(templates_detections, model, sigma=5):
    first_detection_indexes = []
    for template_result in templates_detections.values():
            if len(template_result[model][f"{sigma}s_detections_indexes"]) > 0:
                first_detection_indexes.append(template_result[model][f"{sigma}s_detections_indexes"][0])
    return np.array(sorted(first_detection_indexes))

In [ ]:
lima_first_d = get_first_detections(templates_detections, "li_ma", 5)
ad_first_d = get_first_detections(templates_detections, "rnn", 5)

In [ ]:
lima_first_d

In [ ]:
ad_first_d

In [ ]:
bins = [i for i in range(0,500,1)]

In [ ]:
%matplotlib inline
from rtapipe.lib.plotting.PlotConfig import PlotConfig
pc = PlotConfig()
fig, ax = plt.subplots(1,1,figsize=pc.fig_size)
print(len(lima_first_d), len(ad_first_d))

counts_li_ma, bins_li_ma = np.histogram(lima_first_d, bins=bins)
#ax.stairs(counts_li_ma, bins_li_ma, label="Li&Ma", color="grey", alpha=0.5)
ax.hist(bins_li_ma[:-1], bins_li_ma, weights=counts_li_ma, label="Li&Ma", color="grey", alpha=0.5)

counts_ad, bins_ad = np.histogram(ad_first_d, bins=bins)
#ax.stairs(counts_ad, bins_ad, label="A.D.", color="black", alpha=0.8)
ax.hist(bins_ad[:-1], bins_ad, weights=counts_ad, label="A.D.", color="black", alpha=0.8)

#counts_li_ma, bins_li_ma = ax.hist((np.array(lima_first_d))*5, label="Li&Ma", color="grey", alpha=0.5, bins=100)
#counts_ad, bins_ad       = ax.hist((np.array(ad_first_d))*5, label="A.D.", color="black", alpha=0.8, bins=100)


fig.suptitle("")

#ax.set_xlim(5,50)
ax.set_ylabel("Counts")
ax.set_xlabel("First Detection Time Bin [s-s]")
ax.legend()
ax.set_title("First Detection Time Bin Distrution (Test set E)")
#ax.set_xticks([i*5 +12.5 for i in range(38,94)],sigma_table_rnn.index.values[40:], fontsize=10)
#ax.axvline(x = 227.5, color='black', linestyle="-.", alpha=0.8, label="GRB start")
#ax.text(229, 76, 'GRB start', fontsize = 14, color="black")
for tick in ax.get_xticklabels():
    tick.set_rotation(-90)

ax.set_xlim(240, 300)

ax.tick_params(axis='both', which='major', labelsize=15)
fig.tight_layout()
fig.savefig(Path(output_dir).joinpath(f"ad_vs_li_ma_first_detections_testset_e_id_1.png"), dpi=pc.dpi)

In [ ]:
bins_li_ma

In [ ]:
%matplotlib inline
from rtapipe.lib.plotting.PlotConfig import PlotConfig
pc = PlotConfig()
fig, ax = plt.subplots(1,1,figsize=pc.fig_size)
fig.suptitle("")
ax.set_ylabel("Number of detections")
ax.set_xlabel("Time bin ([s-s])")
ax.set_title("Total number of 5σ detections (Test set E)")
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlim(200, 300)


cumulative_ad = np.cumsum(counts_ad)
ax.plot(bins_ad[:-1], cumulative_ad, marker="*", markersize=2, linestyle="none", label="A.D.", color="black", alpha=0.8)

cumulative_li_ma = np.cumsum(counts_li_ma)
cumulative_li_ma_reduced = []
bins_li_ma_reduced = []
for i in range(2, 500, 1):
    if i % 5 == 0:
        cumulative_li_ma_reduced.append(cumulative_li_ma[i])
        bins_li_ma_reduced.append(bins_li_ma[i])
ax.plot(bins_li_ma_reduced, cumulative_li_ma_reduced, marker="x", markersize=10, linestyle="none", label="Li&Ma", color="grey", alpha=1)

ax.set_xticks([i for i in range(200,496,5)],sigma_table_rnn.index.values[200:], fontsize=5)

for tick in ax.get_xticklabels():
    tick.set_rotation(-90)

#ax.axvline(x = 225, color='black', linestyle="-.", alpha=0.8)    

    
#ax.axvline(x = 275, color='grey', linestyle="dotted", alpha=0.8, ymin=0.08, ymax=0.8)    
#ax.text(262, 15, '<-- A.D.', color='black', fontsize = 8)
#ax.text(278, 15, 'Li&Ma -->', color='black', fontsize = 8)


#ax.text(227, 140, 'GRB start', fontsize = 14, color="red")
#ax.axvspan(225, 280, color='green', alpha=0.1)
#ax.text(227, 175, 'Serendipitous discoveries', color='green', fontsize = 15)
#ax.axvspan(280, 500, color='blue', alpha=0.1)
#ax.text(290, 175, 'Observation triggered by external science alert', color='blue', fontsize = 15)

ax.axhline(y = 188, color='black', linestyle="-.", alpha=0.8, label="Total trials")    

ax.legend()

fig.tight_layout()
fig.savefig(Path(output_dir).joinpath(f"ad_vs_li_cumulative_testset_e_id_1.png"), dpi=pc.dpi)

### 3 sigma detections

In [ ]:
#print(f'Detected by CNN: {np.array([template_result["cnn"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("cnn",templates_detections, 3)} s')
print(f'Detected by RNN: {np.array([template_result["rnn"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("rnn",templates_detections, 3, onset_index=250)} s')
print(f'Detected by Li&Ma: {np.array([template_result["li_ma"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("li_ma",templates_detections, 3, onset_index=250)} s')

In [ ]:
count_rnn = 0
for template_result in templates_detections.values():
    if template_result["rnn"]["3s_detection"] and not template_result["li_ma"]["3s_detection"]:
        count_rnn += 1
print(f"Detected by RNN but not by Li&Ma: {count_rnn}")

In [ ]:
count_rnn = 0
for template_result in templates_detections.values():
    if not template_result["rnn"]["3s_detection"] and template_result["li_ma"]["3s_detection"]:
        count_rnn += 1
print(f"Detected by Li&Ma but not by RNN: {count_rnn}")

### 5 sigma detections

In [ ]:
def get_detections_in_time(templates_detections, model):
    first_detection_indexes = []
    for template_result in templates_detections.values():
        if len(template_result["li_ma"][f"5s_detections_indexes"]) > 0:
            first_detection_indexes.append(template_result["li_ma"][f"5s_detections_indexes"][0])
    detection_time = (np.array(first_detection_indexes)-245)*integration_time
    dt_filterd = [dt for dt in detection_time if dt <= 250]
    return len(dt_filterd)

print(get_detections_in_time(templates_detections, "li_ma"))

In [ ]:
#print(f'Detected by CNN: {np.array([template_result["cnn"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("cnn",templates_detections, 5)} s')
print(f'Detected by RNN: {np.array([template_result["rnn"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("rnn",templates_detections, 5, onset_index=250)} s')
print(f'Detected by Li&Ma: {np.array([template_result["li_ma"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("li_ma",templates_detections, 5, onset_index=250)} s')

In [ ]:
count_cnn = 0
for template_result in templates_detections.values():
    if template_result["cnn"]["5s_detection"] and not template_result["li_ma"]["5s_detection"]:
        count_cnn += 1
print(f"Detected by CNN but not by Li&Ma: {count_cnn}")

In [ ]:
count_rnn = 0
for template_result in templates_detections.values():
    if template_result["rnn"]["5s_detection"] and not template_result["li_ma"]["5s_detection"]:
        count_rnn += 1
print(f"Detected by RNN but not by Li&Ma: {count_rnn}")

In [ ]:
count_rnn = 0
for template_result in templates_detections.values():
    if not template_result["rnn"]["5s_detection"] and template_result["li_ma"]["5s_detection"]:
        count_rnn += 1
print(f"Detected by Li&Ma but not by RNN: {count_rnn}")

### Sigma plots RNN vs Li&Ma

In [ ]:
matplotlib.use("TKAgg", force=True)
%matplotlib inline
from rtapipe.lib.plotting.PlotConfig import PlotConfig

def sigma_plot(template, sigma_table_model, lima_sigma_table):

    cumulative_li_ma_reduced = []
    bins_li_ma_reduced = []
    for i in range(2, 500, 1):
        if i % 5 == 0:
            cumulative_li_ma_reduced.append(lima_sigma_table[template][i])
            bins_li_ma_reduced.append(sigma_table_model.index.values[i])
            
    pc = PlotConfig()    
    fig, ax = plt.subplots(1,1, figsize=pc.fig_size)
    fig.suptitle("Detection plot")
    ax.set_title(f"Template {template}")
    ax.set_ylabel("Sigma")
    ax.set_xlabel("Timeseries")
    ax.set_ylim(0, 10)

    ax.tick_params(axis='y', labelsize=15, width=2, grid_alpha=0.5)
    ax.tick_params(axis='x', which='major', labelsize=10)
    mask = [True if i%10==0 else False for i in range(len(sigma_table_model.index.values))]
    ax.set_xticks(range(0, len(sigma_table_model.index.values), 10), sigma_table_model.index.values[mask], rotation=45)


    ax.plot(sigma_table_model[template], marker='o', markersize=1, linestyle='--', linewidth=0.5, label="Anomaly Detection", zorder=5)
    ax.scatter(bins_li_ma_reduced, cumulative_li_ma_reduced, marker='x', s=100, label="Li&Ma", color="grey", zorder=10)
    plt.axvline(x = 250, color = 'grey', linestyle="-.")
    plt.axhline(y = 3, color = 'orange', linestyle="-.")
    plt.axhline(y = 5, color = 'red', linestyle="-.")

    ax.legend()
    fig.savefig(Path(output_dir).joinpath(f"detection_plot_{template}_testset_e.png"), dpi=100)
    #plt.show()    
    plt.close()

templates = list(data_manager.data.keys())
#templates = ["run0019_ID000070"]    
for template in tqdm(templates):
    sigma_plot(template, sigma_table_rnn, lima_sigma_table)

In [ ]:
len(cumulative_li_ma_reduced)